# Object-Relational Mapper exercise

Creating in-memory SQLite database for the purpose of the exercise

In [9]:
import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import func
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

engine = sa.create_engine('sqlite:///:memory:')
Base = declarative_base()
session = sa.orm.sessionmaker(bind=engine)()

In [10]:
class OrderItem(Base):
    __tablename__ = 'order'

    id = sa.Column('id', sa.Integer, primary_key=True)
    product_id = sa.Column('product_id', sa.Integer, sa.ForeignKey('product.id'))
    user_id = sa.Column('user_id', sa.Integer, sa.ForeignKey('user.id'))
    qty = sa.Column('qty', sa.Integer)

class Product(Base):
    __tablename__ = 'product'

    id = sa.Column('id', sa.Integer, primary_key=True)
    name = sa.Column('name', sa.Text)
    price = sa.Column('price', sa.Numeric(14,2))

class User(Base):
    __tablename__ = 'user'

    id = sa.Column('id', sa.Integer, primary_key=True)
    first_name = sa.Column('first_name', sa.Text)
    last_name = sa.Column('last_name', sa.Text)

Base.metadata.create_all(engine)

engine.echo = True

In [11]:
session.add(User(id=42, first_name='John', last_name='Cleese'))
session.add(User(id=43, first_name='Eric', last_name='Idle'))
session.add(User(id=44, first_name='Terry', last_name='Jones'))
session.add(User(id=13, first_name='Graham', last_name='Chapman'))
session.add(Product(id=1, price=Decimal('2.10'), name='Rabbit'))
session.add(Product(id=2, price=Decimal('5.10'), name='Parrot'))
session.add(OrderItem(product_id=1, user_id=42, qty=9))
session.add(OrderItem(product_id=1, user_id=42, qty=2))
session.add(OrderItem(product_id=1, user_id=13, qty=2))
session.add(OrderItem(product_id=2, user_id=13, qty=3))
session.add(OrderItem(product_id=2, user_id=13, qty=20))
session.commit()

2021-10-15 10:04:10,871 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-15 10:04:10,872 INFO sqlalchemy.engine.Engine INSERT INTO "order" (product_id, user_id, qty) VALUES (?, ?, ?)
2021-10-15 10:04:10,872 INFO sqlalchemy.engine.Engine [generated in 0.00055s] (1, 42, 9)
2021-10-15 10:04:10,873 INFO sqlalchemy.engine.Engine INSERT INTO "order" (product_id, user_id, qty) VALUES (?, ?, ?)
2021-10-15 10:04:10,873 INFO sqlalchemy.engine.Engine [cached since 0.001507s ago] (1, 42, 2)
2021-10-15 10:04:10,873 INFO sqlalchemy.engine.Engine INSERT INTO "order" (product_id, user_id, qty) VALUES (?, ?, ?)
2021-10-15 10:04:10,874 INFO sqlalchemy.engine.Engine [cached since 0.002414s ago] (1, 13, 2)
2021-10-15 10:04:10,874 INFO sqlalchemy.engine.Engine INSERT INTO "order" (product_id, user_id, qty) VALUES (?, ?, ?)
2021-10-15 10:04:10,875 INFO sqlalchemy.engine.Engine [cached since 0.003031s ago] (2, 13, 3)
2021-10-15 10:04:10,875 INFO sqlalchemy.engine.Engine INSERT INTO "order" (product_id,

# Exercise 1.

Taking the database from the example above, can you write ONE sqlalchemy query that will create an outer join on Users, OrderItems and Products that return the last_name, qty and name from the corresponding tables ?

In [12]:
session.query(User.last_name,OrderItem.qty, Product.name).outerjoin(OrderItem, User.id == OrderItem.user_id ).outerjoin(Product, Product.id == OrderItem.product_id ).all()

2021-10-15 10:04:14,002 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-15 10:04:14,004 INFO sqlalchemy.engine.Engine SELECT user.last_name AS user_last_name, "order".qty AS order_qty, product.name AS product_name 
FROM user LEFT OUTER JOIN "order" ON user.id = "order".user_id LEFT OUTER JOIN product ON product.id = "order".product_id
2021-10-15 10:04:14,004 INFO sqlalchemy.engine.Engine [generated in 0.00055s] ()


[('Chapman', 2, 'Rabbit'),
 ('Chapman', 3, 'Parrot'),
 ('Chapman', 20, 'Parrot'),
 ('Cleese', 2, 'Rabbit'),
 ('Cleese', 9, 'Rabbit'),
 ('Idle', None, None),
 ('Jones', None, None)]

## Explanation

We have 3 tables 1.User, 2.Procudt. 3.OrderItem. I'm querying for users's last name, quantity of order and the product name, from the respective tables.

The tables are connected in such way: User > OrderItem < Product
Where The OrderItem has foreign keys: product_id, user_id which are primary keys of Product table and User Table respectively

We can use the above statment to join the tables on that condition to make a Left OUTER JOIN


# Exercise 2.

Taking the database from the example above, can you write ONE sqlalchemy query that will create an aggregated result from the query above (sum the products of qty and price accordingly) calculating the value of purchases (grouping) per User ?

In [13]:
session.query(User.last_name, func.sum(OrderItem.qty * Product.price)).outerjoin(OrderItem, User.id == OrderItem.user_id ).outerjoin(Product, Product.id == OrderItem.product_id ).group_by(User.id).all()

2021-10-15 10:04:16,927 INFO sqlalchemy.engine.Engine SELECT user.last_name AS user_last_name, sum("order".qty * product.price) AS sum_1 
FROM user LEFT OUTER JOIN "order" ON user.id = "order".user_id LEFT OUTER JOIN product ON product.id = "order".product_id GROUP BY user.id
2021-10-15 10:04:16,928 INFO sqlalchemy.engine.Engine [generated in 0.00038s] ()


[('Chapman', Decimal('121.50')),
 ('Cleese', Decimal('23.10')),
 ('Idle', None),
 ('Jones', None)]

## Explanation

The query is quite similar to the one above with just 2 extra conditons:
    1- Group qunantity of Item and Price acording to the user
    2- Sum according -> (order.qty * product.price) 
Using group_by() method to grup the query on User.id
Using func class use python methods on query